In [ ]:
import pandas as pd
import json
import snowflake.connector
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt


#login to snowflake db
con = snowflake.connector.connect(user='vishal.kumar@scale.com',
                                 account='pxa65918',
                                 authenticator='externalbrowser',
                                 warehouse='COMPUTE_WH',
                                 database='SCALE_CRAWLER',
                                 role='GENERAL_RO')

cs = con.cursor()

In [ ]:
#Get json data from snowflake
sql = f'''
select
  result, status, created_at, USER_EMAIL, _ID
from
  PUBLIC.SPOTTERAUDITS
where
  customer = 'woodpecker'
  and cast(created_at as Date) > '2023-05-03'
-- 

'''
cs.execute(sql)
idf = cs.fetch_pandas_all()

In [ ]:
#iterate through each row of idf
for index, row in idf.iterrows():
    #save value of current row json key from first row of RESULT column
    temp = idf['RESULT'][index]
    #convert temp string to dict
    temp = json.loads(temp)
    print(temp)

In [ ]:
def flatten_json(nested_json, parent_key='', sep='.'):
    items = []
    for key, value in nested_json.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            items.extend(flatten_json(value, new_key, sep=sep))
        else:
            items.append({
                'PVID': parent_key.split(sep)[0],
                'Attribute': sep.join(parent_key.split(sep)[1:] + [key]),
                'AuditKey': '',
                'Result': value
            })
    return items

# Flatten the JSON data
rows = flatten_json(temp)

# Create a DataFrame from the flattened data
tdf = pd.DataFrame(rows, columns=['PVID', 'Attribute', 'AuditKey', 'Result'])

#split eeach row in Attribute column by . and save first value in Attribute column and second value in AuditKey column
tdf[['Attribute', 'AuditKey']] = tdf['Attribute'].str.split('.', 1, expand=True)


In [ ]:
#keep only rows where auditkey is result and save it in rdf
rdf = tdf[tdf['AuditKey'] == 'result']
#count % of good values in result column by Attribute
AttScore = rdf.groupby('Attribute')['Result'].value_counts(normalize=True).mul(100).reset_index(name='Score')
#remove rows where result is not good
AttScore = AttScore[AttScore['Result'] == 'Good']
#sort by Score
AttScore = AttScore.sort_values(by=['Score'], ascending=True)
#plot score by attribute in bar chart, make it wide, label x axis vertically
AttScore.plot.bar(x='Attribute', y='Score', rot=90, figsize=(20,10))
#add plot title, axes labels, large font
plt.title('Woodpecker CQR Scores', fontsize=20)
plt.xlabel('Attribute', fontsize=20)
plt.ylabel('Accuracy %', fontsize=20)